In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import os

In [ ]:
dataset_path = os.listdir("/content/drive/MyDrive/Dataset/1000_videos/train")
print(dataset_path)

['fake', 'real']


In [ ]:
class_labels = []

for item in dataset_path:
 # Get all the file names
 all_classes = os.listdir('/content/drive/MyDrive/Dataset/1000_videos/train/' +item)# /content/drive/MyDrive/Dataset/1000_videos/train/ + fake 6000+
                                                                                    # /content/drive/MyDrive/Dataset/1000_videos/train/ + real 5000+
 print(len(all_classes))
 print(all_classes)

 # Add them to the list
 for room in all_classes:   # 11633 images
    class_labels.append((item, str('dataset_path/' +item) + '/' + room))


print(class_labels[:1])
print(class_labels[-1:])
print(len(class_labels))

6028
['id1_id26_0009_0.png', 'id1_id26_0009_1.png', 'id1_id26_0009_10.png', 'id1_id26_0009_11.png', 'id1_id26_0009_12.png', 'id1_id26_0009_13.png', 'id1_id26_0009_2.png', 'id1_id26_0009_3.png', 'id1_id26_0009_4.png', 'id1_id26_0009_5.png', 'id1_id26_0009_6.png', 'id1_id26_0009_7.png', 'id1_id26_0009_8.png', 'id1_id26_0009_9.png', 'id1_id28_0000_0.png', 'id1_id28_0000_1.png', 'id1_id28_0000_10.png', 'id1_id28_0000_11.png', 'id1_id28_0000_2.png', 'id1_id28_0000_3.png', 'id1_id28_0000_4.png', 'id1_id28_0000_5.png', 'id1_id28_0000_6.png', 'id1_id28_0000_7.png', 'id1_id28_0000_8.png', 'id1_id28_0000_9.png', 'id1_id28_0001_0.png', 'id1_id28_0001_1.png', 'id1_id28_0001_2.png', 'id1_id28_0001_3.png', 'id1_id28_0001_4.png', 'id1_id28_0001_5.png', 'id1_id28_0001_6.png', 'id1_id28_0001_7.png', 'id1_id28_0001_8.png', 'id1_id28_0002_0.png', 'id1_id28_0002_1.png', 'id1_id28_0002_10.png', 'id1_id28_0002_11.png', 'id1_id28_0002_12.png', 'id1_id28_0002_2.png', 'id1_id28_0002_3.png', 'id1_id28_0002_4.pn

In [ ]:
# Build a dataframe
df = pd.DataFrame(data=class_labels, columns=['Labels', 'image'])
print(df.head())
print(df.tail())

  Labels                                   image
0   fake   dataset_path/fake/id1_id26_0009_0.png
1   fake   dataset_path/fake/id1_id26_0009_1.png
2   fake  dataset_path/fake/id1_id26_0009_10.png
3   fake  dataset_path/fake/id1_id26_0009_11.png
4   fake  dataset_path/fake/id1_id26_0009_12.png
      Labels                         image
11628   real   dataset_path/real/159_1.png
11629   real  dataset_path/real/159_10.png
11630   real  dataset_path/real/159_11.png
11631   real  dataset_path/real/159_12.png
11632   real   dataset_path/real/159_2.png


In [ ]:
# Let's check how many samples for each category are present
print("Total number of images in the dataset: ", len(df))

label_count = df['Labels'].value_counts()
print(label_count)

Total number of images in the dataset:  11633
Labels
fake    6028
real    5605
Name: count, dtype: int64


In [ ]:
import cv2
path = '/content/drive/MyDrive/Dataset/1000_videos/train/'
dataset_path = os.listdir('/content/drive/MyDrive/Dataset/1000_videos/train')

im_size = 224

images = []
labels = []

for i in dataset_path:
    data_path = path + str(i)  # /content/drive/MyDrive/Dataset/1000_videos/train/fake 5000+
                               # /content/drive/MyDrive/Dataset/1000_videos/train/real 6000+
    filenames = [i for i in os.listdir(data_path) ]

    for f in filenames:
        img = cv2.imread(data_path + '/' + f)
        img = cv2.resize(img, (im_size, im_size))
        images.append(img)
        labels.append(i)

In [ ]:
#This model takes input images of shape (224, 224, 3), and the input data should range [0, 255].

images = np.array(images)

images = images.astype('float32') / 255.0
images.shape

(11633, 224, 224, 3)

In [ ]:
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
y=df['Labels'].values
print(y)

y_labelencoder = LabelEncoder ()
y = y_labelencoder.fit_transform (y)
print (y)

['fake' 'fake' 'fake' ... 'real' 'real' 'real']
[0 0 0 ... 1 1 1]


In [ ]:
y=y.reshape(-1,1)

from sklearn.compose import ColumnTransformer
ct = ColumnTransformer([('my_ohe', OneHotEncoder(), [0])], remainder='passthrough')
Y = ct.fit_transform(y) #.toarray()
print(Y[:5])
print(Y[35:])

[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [ ]:
from sklearn.utils import shuffle


images, Y = shuffle(images, Y, random_state=1)


train_x, train_y = images, Y

#inpect the shape of the training and testing.
print(train_x.shape)
print(train_y.shape)

(11633, 224, 224, 3)
(11633, 2)


In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB0

NUM_CLASSES = 2
IMG_SIZE = 224
size = (IMG_SIZE, IMG_SIZE)


inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))


# Using model without transfer learning

outputs = EfficientNetB0(include_top=True, weights=None, classes=NUM_CLASSES)(inputs)

In [ ]:
model = tf.keras.Model(inputs, outputs)
model.compile(
   optimizer=tf.keras.optimizers.Adam(),
   loss='binary_crossentropy',
   metrics=['binary_accuracy']
)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ efficientnetb0 (Functional)          │ (None, 2)                   │       4,052,133 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,052,133 (15.46 MB)

 Trainable params: 4,010,110 (15.30 MB)

 Non-trainable params: 42,023 (164.16 KB)

In [ ]:
hist = model.fit(train_x, train_y, epochs=1, batch_size=32)

NotFoundError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever

  File "/usr/lib/python3.11/asyncio/base_events.py", line 1936, in _run_once

  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run

  File "/usr/local/lib/python3.11/dist-packages/tornado/ioloop.py", line 699, in <lambda>

  File "/usr/local/lib/python3.11/dist-packages/tornado/ioloop.